<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/%E6%83%85%E5%A0%B1%E8%AA%8D%E8%AD%98%E8%AB%962021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 情報認識論試験問題

以下の問題について解答し、結果を提出してください。なお、解答については文章はテキストセルに、プログラムに関してはコードセルに記入すること。また、結果の提出については、この画面の左上の「ファイル」メニュー内にある「ダウンロード」->「.ipynbをダウンロード」で自分のマシンにダウンロードできるので、それを各自の学内メールに添付してください。

#1. ニューラルネット

(1) ニューラルネットを学習させる際に用いられる「誤差逆伝播法」について説明せよ。

【解答欄】

(2) 誤差逆伝播法では、微分操作が必要となる。以下のネットワークを考える。

- 入力層：$x = (x_1, x_2, ..., x_i, ..., x_n)$
- 第１層：$y = x + 2$
- 第２層：$z = y \odot y \times 3$ (注：$\odot$ はアダマール積（成分ごとの積）)
- 出力層：$out = mean(z)$ （$z_i$の平均値）

このとき、出力値に対する入力値の微分
$\frac{\partial}{\partial x_{i}}out$ を計算せよ。

【解答欄】
$$ \frac{\partial}{\partial x_{i}}out = ここに数式を書く $$

(3) $x = (1, 2, 3, 4)$ となった時の、$\frac{\partial}{\partial x}out$ を求めよ。





【解答欄】
$$
(\frac{\partial}{\partial x_1}out, \frac{\partial}{\partial x_2}out, \frac{\partial}{\partial x_3}out, \frac{\partial}{\partial x_4}out) = ここに数値を書く
$$

# 2. Pytorch

(1) 本講義では、ニューラルネットの実装に Pytorch を用いたが、そこで用いられる基本的なデータ構造に「テンソル」がある。テンソルとはどのようなもので、ニューラルネットにおいてはどのように使われるかを説明せよ。

【解答欄】

(2) 2 x 2 のリスト [[1, 2], [3, 4]] からテンソルを作り、x という変数に代入せよ。

（ヒント：テンソルの作成には、torch.Tensor() を用いる）

In [38]:
import torch

l = [[1, 2], [3, 4]]         # ここにリストを書く
print("list: ", l)

x = torch.Tensor(list)         # ここでテンソルに変換
print("x: ", x)

list:  [[1, 2], [3, 4]]
x:  tensor([[1., 2.],
        [3., 4.]])


Pytorch には微分を自動で行なってくれるため、上記の誤差逆伝播法が簡単に実行できる。そのためには、テンソルの requires_grad 属性を True にしておく必要がある。


In [39]:
x.requires_grad = True # これはそのまま実行してください。

(3) 問１でのネットワークをテンソル x を用いて構築せよ。

In [40]:
y = x + 2# ここを自分で埋める
z = y * y * 3# ここも自分で埋める埋める
out = z.mean() # ここも自分で埋める

(4) backword()関数を用いて、out に対する x の微分値を求めよ。

In [41]:
out.backward() # これで逆伝播を行う
x.grad # x の grad 属性に微分値が格納される

tensor([[4.5000, 6.0000],
        [7.5000, 9.0000]])

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))    

tensor([[1.1802e+26, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00]], device='cuda:0')
tensor([[1.1802e+26, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00]], dtype=torch.float64)
